In [106]:
import numpy as np


def gen_pow_matrix(primpoly):
    deg = len(bin(primpoly)) - 3
    max_dec = 2 ** deg
    solve_poly = primpoly % max_dec
    degs = np.zeros(max_dec - 1, dtype=int)
    cur_poly = 2   # cur_poly = x
    degs[0] = cur_poly
    for i in range(1, max_dec - 1):
        res = cur_poly * 2  # mul on x == shift left on 1 bit
        if res >= max_dec:
            res = res % max_dec
            res = res ^ solve_poly
        cur_poly = res
        degs[i] = cur_poly
    pos, values = np.unique(degs, return_index=True)
    matrix = np.zeros((max_dec - 1, 2), dtype=int)
    matrix[pos - 1, 0] = values + 1
    matrix[:, 1] = degs
    return matrix


def add(X, Y):
    return X ^ Y


def sum(X, axis=0):
    return np.bitwise_xor.reduce(X, axis=axis)


def prod(X, Y, pm):
    def prod_one(x, y):
        if x == 0 or y == 0:
            return 0
        x_pow = pm[x - 1, 0]
        y_pow = pm[y - 1, 0]
        res = (x_pow + y_pow) % pm.shape[0]
        return pm[res - 1, 1]
    return np.vectorize(prod_one)(X, Y)


def divide(X, Y, pm):
    def divide_one(x, y):
        assert y != 0
        if x == 0:
            return 0
        x_pow = pm[x - 1, 0]
        y_pow = pm[y - 1, 0]
        res = (x_pow - y_pow) % pm.shape[0]
        return pm[res - 1, 1]
    return np.vectorize(divide_one)(X, Y)

def linsolve(A, b, pm):
    matrix = np.hstack((A, b[:, np.newaxis]))
    for i in range(A.shape[0] - 1):
        matrix_part = matrix[i:, i:]
        if (np.all(matrix_part[:, 0] == 0)):
            return np.nan
        if matrix_part[0, 0] == 0:
            non_zero = np.where(matrix_part[:, 0] != 0)[0][0]
            matrix_part[0], matrix_part[non_zero] = matrix_part[non_zero], matrix_part[0].copy()
        arr1 = matrix_part[0][np.newaxis, :]
        arr2 = matrix_part[1:, 0][:, np.newaxis]
        mul = prod(arr1, divide(arr2, matrix_part[0][0], pm), pm)
        matrix_part[1:] = add(matrix_part[1:], mul)
    if matrix[-1, -2] == 0:
        return np.nan
    res = np.zeros(A.shape[0], dtype=np.int64)
    for i in range(A.shape[0], 0, -1):
        matrix_part = matrix[:i, :i + 1]
        numer = matrix_part[-1, -1]
        denom = matrix_part[-1, -2]
        root = divide(numer, denom, pm)
        res[i - 1] = root
        matrix_part[:, -2] = add(matrix_part[:, -1], prod(matrix_part[:, -2],
                                root, pm))
    return res

def minpoly(x, pm):
    roots = set()
    for elem in x:
        #if deg == -1:
        #    continue
        #if deg in roots:
        #    continue
        cycle = set()
        cycle.add(elem)
        new = int(prod(elem, elem, pm))
        #tmp = (deg * 2) % pm.shape[0]
        while new not in cycle:
            cycle.add(new)
            new = int(prod(new, new, pm))
        roots.update(cycle)
    roots = np.array(list(roots))
    roots = np.sort(roots)
    res = np.array([1, roots[0]])
    for i, root in enumerate(roots[1:]):
        res = polyprod(res, np.array([1, root]), pm)
    return res, roots

def polyval(p, x, pm):
    res = np.zeros_like(x)
    cur = 1
    for i, koef in enumerate(list(p)[::-1]):
        value = prod(cur, koef, pm)
        res = res ^ value
        cur = prod(cur, x, pm)
    return res

def normalize_poly(p):
    for i in range(p.shape[0]):
        if p[i] != 0:
            break
    return p[i:]

def polyprod(p1, p2, pm):
    p1_pow = pm[p1 - 1, 0]
    p2_pow = pm[p2 - 1, 0]
    res = np.zeros(p1.shape[0] + p2.shape[0] - 1, dtype=np.int64)
    for i1, pow1 in enumerate(p1_pow[::-1]):
        for i2, pow2 in enumerate(p2_pow[::-1]):
            if p1[::-1][i1] == 0 or p2[::-1][i2] == 0:
                continue
            res_pow = (pow1 + pow2) % pm.shape[0]
            res[i1 + i2] = res[i1 + i2] ^ pm[:, 1][res_pow - 1]             
    return normalize_poly(res[::-1])

def polydivmod(p1, p2, pm):
    p1 = normalize_poly(p1)
    p2 = normalize_poly(p2)
    max_deg = p2.shape[0] - 1
    cur_poly = p1
    cur_deg = p1.shape[0] - 1
    if cur_deg < max_deg:
        return np.array([0]), p1
    res = np.zeros(cur_deg - max_deg + 1, dtype = np.int64)
    while cur_deg >= max_deg:
        koef = int(divide(cur_poly[0], p2[0], pm))
        res[cur_deg - max_deg] = koef
        tmp_poly = np.zeros(cur_deg - max_deg + 1, dtype = np.int64)
        tmp_poly[0] = koef
        tmp_poly = polyprod(tmp_poly, p2, pm)
        cur_poly = cur_poly ^ tmp_poly
        flag = True
        for i in range(cur_poly.shape[0]):
            if cur_poly[i] != 0:
                index = i
                flag = False
                break
        if flag:
            break
        cur_poly = cur_poly[index:]
        cur_deg = cur_poly.shape[0] - 1
    return normalize_poly(res[::-1]), normalize_poly(cur_poly)

def polyadd(p1, p2):
    if len(p1) > len(p2):
        p1, p2 = p2, p1.copy()
    p1 = np.concatenate([np.zeros(len(p2) - len(p1), dtype=np.int64), p1])
    res = p1 ^ p2
    return normalize_poly(res)

def euclid(p1, p2, pm, max_deg=0):
    p1_deg = p1.shape[0] - 1
    p2_deg = p2.shape[0] - 1
    if p1_deg < p2_deg:
        p1, p2 = p2, p1.copy()
        p1_deg, p2_deg = p2_deg, p1_deg
    r_old = p1
    r_new = p2
    x_old = np.array([1])
    x_new = np.array([0])
    y_old = np.array([0])
    y_new = np.array([1])
    deg_new = p2_deg
    while deg_new > max_deg:
        q, r = polydivmod(r_old, r_new, pm)
        r_old = r_new
        r_new = r
        deg_new = r_new.shape[0] - 1
        x = polyadd(x_old, polyprod(q, x_new, pm))
        x_old = x_new
        x_new = x
        y = polyadd(y_old, polyprod(q, y_new, pm))
        y_old = y_new
        y_new = y
    r_new = normalize_poly(r_new)
    x_new = normalize_poly(x_new)
    y_new = normalize_poly(y_new)
    return r_new, x_new, y_new

In [97]:
pm = gen_pow_matrix(11)
x = np.array([2])
minpoly(x, pm)

(array([1, 0, 1, 1]), array([2, 4, 6]))

In [98]:
pm = gen_pow_matrix(59)
p = np.arange(6)[::-1]
x = np.array([0, 25, 1, 19, 27, 9, 13, 11, 31, 14, 2, 6, 4, 17, 5,
                 10, 20, 30, 3, 15, 7, 28, 23, 22, 21, 16, 18, 12, 26])
polyval(p, x, pm)

array([ 0, 28,  1, 26, 13, 13, 21,  6, 27, 16, 30,  5, 20,  3, 14, 28, 30,
       19,  2, 17,  2, 16, 27, 21, 13, 11, 15,  9, 30])

In [99]:
pm = gen_pow_matrix(19)
p1 = np.array([pm[5, 1], pm[-1, 1]])
zero = np.array([0])
polyprod(p1, zero, pm)

array([0])

In [100]:
div = polydivmod(np.array([0b10, 0b1]), np.array([0b1]), gen_pow_matrix(0b1011))
div

(array([2, 1]), array([0]))

In [101]:
pm = gen_pow_matrix(19)
x = np.zeros(16, dtype=np.int64)
x[0] = 1
x[-1] = 1
y = np.array([1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1])
print(polydivmod(x, y, pm))

(array([1, 0, 1, 0, 1, 1]), array([0]))


In [103]:
pm = gen_pow_matrix(37)
p1 = np.array([2, 14, 22, 23, 8, 17, 1, 11, 26, 3])
p2 = np.array([31, 23, 29, 31, 11, 9])
result = euclid(p1, p2, pm, max_deg=3)
result

1
1


(array([20, 18, 20, 29]), array([0]), array([ 9, 17, 14,  6,  8, 15]))

In [107]:
pm = gen_pow_matrix(37)
p1 = np.array([2, 14, 22, 23, 8, 17, 1, 11, 26, 3])
p2 = np.array([31, 23, 29, 31, 11, 9])
result = euclid(p1, p2, pm, max_deg=3)
result

(array([20, 18, 20, 29]), array([14,  3]), array([ 9, 16, 15,  7,  9, 15]))

In [108]:
polyadd(polyprod(p1, result[1], pm), polyprod(p2, result[2], pm))

array([20, 18, 20, 29])